In [16]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy

import gc



nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

# Get Data

## Changes from year to year

In [18]:
changes = pd.read_csv("../../data/companyData/compustatChanges_all.csv").drop(columns = ['Unnamed: 0'])

changes.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,priceCloseLast,cashLast,incomeChange,revenueChange,costChange,inventoryChange,opInc_afDepChange,opInc_befDepChange,priceCloseChange,assetsPrev
0,1004,20000229,1999,3.0,AAR CORP,USD,1129.147545,NaN,333.569319,477.771178,...,22.960362,NaN,0.051819,0.070750,0.077057,0.139125,0.077069,0.074228,0.549549,1075.103596
1,1082,20000131,1999,3.0,SERVIDYNE INC,USD,165.794462,NaN,39.497057,17.258803,...,10.436528,NaN,2.519227,0.062572,0.087773,-0.003145,0.173010,-0.019499,-0.497620,168.611793
2,1244,20000229,1999,3.0,ALCIDE CORP,USD,21.983589,NaN,2.067281,2.405836,...,23.529628,NaN,-0.962616,0.244343,0.275102,-0.466926,0.666626,-2.373544,0.329021,27.689058
3,1258,20000229,1999,3.0,CAPCO ENERGY INC,USD,41.520895,NaN,3.947309,1.427623,...,NaN,NaN,-1.141501,3.207090,inf,inf,-1.249781,-1.245783,NaN,1.316141
4,1331,20000131,1999,3.0,ALPINE GROUP INC,USD,NaN,NaN,NaN,NaN,...,21.157689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.110536,3716.041664


In [19]:
otherControls = pd.read_csv('../../data/companyData/otherControls.csv').\
    drop(columns = {'Unnamed: 0', 'fyearq'}).rename(columns = {'year_toMatchOn': 'year',
                                                              'fqtr': 'qtr'})

otherControls.head()

,gvkey,qtr,assetsLagged,netIncomeLagged,roa_lagged,famafrench,sic2,indGroup,year,earliestYear,ageTercile,sizeTercile,profitTercile
0,1010,1.0,3422.936770,46.757990,0.013660,25.0,37,manu,1998,1962,0,2.0,2.0
1,1010,2.0,3454.132166,18.500640,0.005356,25.0,37,manu,1998,1962,0,2.0,1.0
2,1010,3.0,3487.709411,10.470317,0.003002,25.0,37,manu,1998,1962,0,2.0,1.0
3,1010,4.0,3448.667709,68.705992,0.019922,25.0,37,manu,1998,1962,0,2.0,2.0
4,1010,1.0,4322.135119,260.987674,0.060384,25.0,37,manu,1999,1962,0,2.0,2.0


In [20]:
otherControls.head()

,gvkey,qtr,assetsLagged,netIncomeLagged,roa_lagged,famafrench,sic2,indGroup,year,earliestYear,ageTercile,sizeTercile,profitTercile
0,1010,1.0,3422.936770,46.757990,0.013660,25.0,37,manu,1998,1962,0,2.0,2.0
1,1010,2.0,3454.132166,18.500640,0.005356,25.0,37,manu,1998,1962,0,2.0,1.0
2,1010,3.0,3487.709411,10.470317,0.003002,25.0,37,manu,1998,1962,0,2.0,1.0
3,1010,4.0,3448.667709,68.705992,0.019922,25.0,37,manu,1998,1962,0,2.0,2.0
4,1010,1.0,4322.135119,260.987674,0.060384,25.0,37,manu,1999,1962,0,2.0,2.0


In [21]:
print(changes.shape)
changes = changes.merge(otherControls)
print(changes.shape)


industries = changes[['gvkey','indGroup']].drop_duplicates()

(891345, 37)
(827747, 47)


In [22]:
industries.to_csv("../../data/companyData/gvkeyIndustries.csv")

In [23]:
changes.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changes.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,assetsLagged,netIncomeLagged,roa_lagged,famafrench,sic2,indGroup,earliestYear,ageTercile,sizeTercile,profitTercile
0,1004,20000229,1999,3.0,AAR CORP,USD,1129.147545,NaN,333.569319,477.771178,...,1016.987327,14.301036,0.014062,41.0,50,wholesale,1968,0,2.0,2.0
1,1082,20000131,1999,3.0,SERVIDYNE INC,USD,165.794462,NaN,39.497057,17.258803,...,172.762477,0.064488,0.000373,42.0,87,services,1981,0,1.0,1.0
2,1244,20000229,1999,3.0,ALCIDE CORP,USD,21.983589,NaN,2.067281,2.405836,...,28.175222,1.368072,0.048556,14.0,28,manu,1983,0,0.0,2.0
3,1258,20000229,1999,3.0,CAPCO ENERGY INC,USD,41.520895,NaN,3.947309,1.427623,...,0.076772,-0.004606,-0.060000,30.0,13,mining,1981,0,0.0,0.0
4,1331,20000131,1999,3.0,ALPINE GROUP INC,USD,NaN,NaN,NaN,NaN,...,1163.929939,2.029844,0.001744,19.0,33,manu,1981,0,2.0,1.0


In [24]:
changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = {'Unnamed: 0'})
changes.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,assetsLagged,netIncomeLagged,roa_lagged,famafrench,sic2,indGroup,earliestYear,ageTercile,sizeTercile,profitTercile
0,1004,20000229,1999,3.0,AAR CORP,USD,1129.147545,NaN,333.569319,477.771178,...,1016.987327,14.301036,0.014062,41.0,50,wholesale,1968,0,2.0,2.0
1,1082,20000131,1999,3.0,SERVIDYNE INC,USD,165.794462,NaN,39.497057,17.258803,...,172.762477,0.064488,0.000373,42.0,87,services,1981,0,1.0,1.0
2,1244,20000229,1999,3.0,ALCIDE CORP,USD,21.983589,NaN,2.067281,2.405836,...,28.175222,1.368072,0.048556,14.0,28,manu,1983,0,0.0,2.0
3,1258,20000229,1999,3.0,CAPCO ENERGY INC,USD,41.520895,NaN,3.947309,1.427623,...,0.076772,-0.004606,-0.060000,30.0,13,mining,1981,0,0.0,0.0
4,1331,20000131,1999,3.0,ALPINE GROUP INC,USD,NaN,NaN,NaN,NaN,...,1163.929939,2.029844,0.001744,19.0,33,manu,1981,0,2.0,1.0


Put in the calendar quarters and fiscal quarter data.

In [25]:
quarters = pd.read_csv("../../data/companyData/fiscalYears.csv")
quarters.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,curcdq,datacqtr,datafqtr,costat,fyrc
0,1004,19980228,1997,3.0,5,INDL,C,D,STD,AIR,USD,1998Q1,1997Q3,A,5
1,1004,19980531,1997,4.0,5,INDL,C,D,STD,AIR,USD,1998Q2,1997Q4,A,5
2,1004,19980831,1998,1.0,5,INDL,C,D,STD,AIR,USD,1998Q3,1998Q1,A,5
3,1004,19981130,1998,2.0,5,INDL,C,D,STD,AIR,USD,1998Q4,1998Q2,A,5
4,1004,19990228,1998,3.0,5,INDL,C,D,STD,AIR,USD,1999Q1,1998Q3,A,5


In [26]:
len(quarters.gvkey.unique())

27703

In [27]:
sum((quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9))/quarters.shape[0]

0.8661478661478661

In [28]:
quarters = quarters[(quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9)][['gvkey','datadate','datacqtr','datafqtr','fyr']].reset_index(drop = True)


In [29]:
quarters.head()

,gvkey,datadate,datacqtr,datafqtr,fyr
0,1010,19980331,1998Q1,1998Q1,12
1,1010,19980630,1998Q2,1998Q2,12
2,1010,19980930,1998Q3,1998Q3,12
3,1010,19981231,1998Q4,1998Q4,12
4,1010,19990331,1999Q1,1999Q1,12


Merge the quarter data into the change data, and make sure that the quarters that are used line up with the calendar quarters.

In [30]:
changesCal = changes[changes.gvkey.isin(quarters.gvkey.unique())]

changesCal = changesCal.merge(quarters)

print(changesCal.shape[0]/changes.shape[0])

0.7815951009185174


In [31]:
changesCal.loc[~(changesCal.datacqtr.isna()), 'year'] = changesCal.datacqtr.str.slice(0,4)
changesCal.loc[~(changesCal.datacqtr.isna()), 'qtr']  = changesCal.datacqtr.str.slice(5,6)

changesCal['DATE'] = pd.to_datetime(changesCal['datadate'])

changesCal.loc[(changesCal.datacqtr.isna()), 'year'] = changesCal.DATE.dt.year
changesCal.loc[(changesCal.datacqtr.isna()), 'qtr']  = changesCal.DATE.dt.quarter

changesCal['year'] = changesCal.year.astype('int64')
changesCal['qtr']  = changesCal.qtr.astype('int64')

print(changesCal.shape,changesCal.head())

(646963, 51)    gvkey  datadate  year  qtr              companyName curcdq      assets  \
0   8515  19990930  1999    3                  PHI INC    USD         NaN   
1  12405  19990930  1999    3               AVESIS INC    USD         NaN   
2  24474  19990930  1999    3  TECHNOLOGY SOLUTIONS CO    USD  362.960666   
3  26830  19990930  1999    3   NAMIBIAN MINERALS CORP    USD  111.299129   
4  30448  19990331  1970    1   AMERICAN HOMESTAR CORP    USD         NaN   

   cash  costGoodsSold  totalInv  ...  sic2            indGroup  earliestYear  \
0   NaN            NaN       NaN  ...    45  transportUtilities          1981   
1   NaN       2.470251       NaN  ...    64             finance          1984   
2   NaN      36.229854  0.000000  ...    73            services          1990   
3   NaN       7.938061  4.269385  ...    14              mining          1994   
4   NaN            NaN       NaN  ...    24                manu          1993   

   ageTercile sizeTercile  profitTerc

In [32]:
changesCal.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,sic2,indGroup,earliestYear,ageTercile,sizeTercile,profitTercile,datacqtr,datafqtr,fyr,DATE
0,8515,19990930,1999,3,PHI INC,USD,NaN,NaN,NaN,NaN,...,45,transportUtilities,1981,0,1.0,1.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
1,12405,19990930,1999,3,AVESIS INC,USD,NaN,NaN,2.470251,NaN,...,64,finance,1984,0,0.0,2.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
2,24474,19990930,1999,3,TECHNOLOGY SOLUTIONS CO,USD,362.960666,NaN,36.229854,0.000000,...,73,services,1990,0,1.0,2.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
3,26830,19990930,1999,3,NAMIBIAN MINERALS CORP,USD,111.299129,NaN,7.938061,4.269385,...,14,mining,1994,1,0.0,0.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
4,30448,19990331,1970,1,AMERICAN HOMESTAR CORP,USD,NaN,NaN,NaN,NaN,...,24,manu,1993,0,1.0,2.0,NaN,1999Q3,6,1970-01-01 00:00:00.019990331


In [ ]:
changesCal = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

# Compustat and ABI Linking

In [33]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns



hasMatch = gvKey_abiLinkingTable.gvkey.unique()

gvKey_abiLinkingTable.head()


,cstatCompanies,igCompanies,delete,gvkey,abi
0,asa gold and precious metals,asa gold precious metals,NaN,1062,402180222
1,adams diversified equity fd,adams diversified equity fund,NaN,1119,397759739
2,allen organ,allen organ,NaN,1283,400700704
3,american physicians svc gp,american physicians svc,NaN,1539,218548014
4,american science engineering,american science engineering,NaN,1554,441435880


---------------------------------

# Get all change data together
Get the linking table and merge the abi labels into the change df. 

Then, merge the location data into the change data and get as complete a record of companies as possible given the HQ data.

In [34]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.shape, changes.head())


changesABI = changes.merge(gvKey_abiLinkingTable, on ='gvkey').drop(columns = {'state','city'})
print(changesABI.shape, changesABI.head())

(646963, 51)    gvkey  datadate  year  qtr              companyName curcdq      assets  \
0   8515  19990930  1999    3                  PHI INC    USD         NaN   
1  12405  19990930  1999    3               AVESIS INC    USD         NaN   
2  24474  19990930  1999    3  TECHNOLOGY SOLUTIONS CO    USD  362.960666   
3  26830  19990930  1999    3   NAMIBIAN MINERALS CORP    USD  111.299129   
4  30448  19990331  1970    1   AMERICAN HOMESTAR CORP    USD         NaN   

   cash  costGoodsSold  totalInv  ...  sic2            indGroup  earliestYear  \
0   NaN            NaN       NaN  ...    45  transportUtilities          1981   
1   NaN       2.470251       NaN  ...    64             finance          1984   
2   NaN      36.229854  0.000000  ...    73            services          1990   
3   NaN       7.938061  4.269385  ...    14              mining          1994   
4   NaN            NaN       NaN  ...    24                manu          1993   

   ageTercile sizeTercile  profitTerc

Now merge in the hq information.

In [35]:
canadian = ['ON', 'AB','QC', 'BC', 'NS', 'NF', 'SK', 'MB', 'NB']
changes = changes[~(changes.state.isin(canadian)) & ~changes.state.isna()]

changes['addzip'] = changes.addzip.astype('str').str.slice(0,5)

changes.state.unique()

array(['LA', 'AZ', 'IL', 'TX', 'IA', 'GA', 'CA', 'PA', 'MO', 'SC', 'NC',
       'UT', 'NV', 'MA', 'MN', 'OH', 'CO', 'FL', 'NJ', 'NY', 'WI', 'IN',
       'VA', 'MD', 'CT', 'MI', 'KS', 'TN', 'DE', 'WA', 'OR', 'HI', 'KY',
       'RI', 'ME', 'NM', 'VT', 'AL', 'AR', 'ID', 'WY', 'OK', 'PR', 'NE',
       'SD', 'DC', 'WV', 'MS', 'ND', 'MT', 'NH', 'AK', 'GU', 'VI'],
      dtype=object)

In [38]:
hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

igChanges = changesABI.merge(hq)
print(igChanges.shape, igChanges.head())


hq.head()

(241355, 64)    gvkey  datadate  year  qtr companyName curcdq      assets  cash  \
0   8515  20030331  2003    1     PHI INC    USD  524.003551   NaN   
1   8515  20030630  2003    2     PHI INC    USD  510.945781   NaN   
2   8515  20030930  2003    3     PHI INC    USD  518.038412   NaN   
3   8515  20031231  2003    4     PHI INC    USD  518.515573   NaN   
4   8515  20040331  2004    1     PHI INC    USD  520.557973   NaN   

   costGoodsSold   totalInv  ...                    company  state       city  \
0      67.936692  52.543325  ...  PETROLEUM HELICOPTERS INC     LA  LAFAYETTE   
1      67.929738  53.962642  ...  PETROLEUM HELICOPTERS INC     LA  LAFAYETTE   
2      73.764198  55.502832  ...  PETROLEUM HELICOPTERS INC     LA  LAFAYETTE   
3      74.286580  55.505099  ...  PETROLEUM HELICOPTERS INC     LA  LAFAYETTE   
4      68.988050  53.825001  ...  PETROLEUM HELICOPTERS INC     LA  LAFAYETTE   

            address_line_1 zipcode  latitude longitude  \
0  2001 SE EVANGELINE

,abi,ticker,company,year,state,city,address_line_1,zipcode,latitude,longitude,parent_employee_size_code,location_employee_size_code,employeesAtLocation
0,7609,SODI,SOLITRON DEVICES INC,2003,FL,WEST PALM BEACH,3301 ELECTRONICS WAY # C,33407,26.74120,-80.06694,NaN,NaN,NaN
1,23077,NaN,JENNY LEE BAKERY,2003,PA,MC KEES ROCKS,620 ISLAND AVE,15136,40.47235,-80.06152,50.0,50.0,1.0
2,76547,NaN,MASTER PROTECTION CORP,2003,FL,FORT MYERS,12800 UNIVERSITY DR # 400,33907,26.55504,-81.88423,20.0,20.0,1.0
3,77743,NaN,NATIONAL TECHNICAL SYSTEMS INC,2003,CA,CALABASAS,24007 VENTURA BLVD # 200,91302,34.15562,-118.65163,10.0,10.0,1.0
4,89151,NaN,HILLTOP BASIC RESOURCES,2003,OH,CINCINNATI,1 W 4TH ST # 1100,45202,39.09982,-84.51297,20.0,20.0,1.0


In [39]:
igChanges.to_csv("../../data/companyData/igData.csv")

In [29]:
igChanges.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue', 'cik',
       'costat', 'add1', 'add2', 'addzip', 'sic', 'assetsLast',
       'netIncomeLast', 'totalRevenueLast', 'costGoodsSoldLast',
       'totalInvLast', 'incomeChange', 'revenueChange', 'costChange',
       'inventoryChange', 'assetsPrev', 'assetsLagged', 'netIncomeLagged',
       'roa_lagged', 'famafrench', 'sic2', 'indGroup', 'earliestYear',
       'ageTercile', 'sizeTercile', 'profitTercile', 'datacqtr', 'datafqtr',
       'fyr', 'DATE', 'cstatCompanies', 'igCompanies', 'delete', 'abi',
       'ticker', 'company', 'state', 'city', 'address_line_1', 'zipcode',
       'latitude', 'longitude', 'parent_employee_size_code',
       'location_employee_size_code', 'employeesAtLocation'],
      dtype='object')

At this point, we have zip information in the following forms (from most to least examples):
    - changes: all compustat companies, from the compustat address system
    - igChanges: subset of compustat companies, from the ig merge
    - subset of compustat companies that have SC information and survived the ig merge
    
We could potentially look at the subset of compustat companies for which we have SC information, usign the compustat address system as well.

For now: follow similar trajectory as before but add in weather data for all cstat companies and all ig-merged companies.

First: pull all zips that are mentioned in changes and igChanges and use this to get the weather data.



In [30]:
changes = changes[(~changes.addzip.isna()) & (changes.addzip != 'nan')]
relevantZips = changes.addzip.astype('int64').append(igChanges.zipcode).unique()

changes.rename(columns = {'addzip': 'zipcode'}, inplace = True)
changes.drop(columns = {'cik',
     'datadate','costat', 'add1', 'add2', 'city', 'sic', 'state'}, inplace = True)

In [31]:
len(relevantZips)

4538

In [ ]:
# relevantZips = allCustomerData.zipcode.append(allSupplierData.zipcode).unique()
outfile =  '../../data/companyData/relevantZips.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(relevantZips, pickle_file)

------------------------------------------------

# Stocks

In [ ]:
igChanges = pd.read_csv("../../data/companyData/igData.csv").\
    drop(columns = {'Unnamed: 0'})
igChanges.head()

In [ ]:
with open('../../data/stockReturns.pkl', 'rb') as f:
    stocks = pkl.load(f)[['date','gvkey','RET']]

In [ ]:
stocks.head()

In [ ]:
sum(stocks.gvkey.isna())

In [ ]:
stocks = stocks[stocks.date.dt.year > 2008]

stocks['qtr']  = stocks.date.dt.quarter
stocks['year'] = stocks.date.dt.year

stocks = stocks[~stocks.gvkey.isna()]
stocks['gvkey'] = stocks['gvkey'].astype(int)
stocks.shape

In [ ]:
igChanges.columns

In [ ]:
companyControls = igChanges[['gvkey','year','qtr','famafrench','ageTercile','sizeTercile','profitTercile','zipcode']]
companyControls.head()

In [ ]:
print(stocks.dtypes, companyControls.dtypes)

In [ ]:
stocksWithControls = stocks.merge(companyControls)
print(stocksWithControls.shape,stocks.shape,companyControls.shape)
stocksWithControls.head()

In [ ]:
del stocks
del companyControls
del igChanges
gc.collect()

In [ ]:
annualWeather = pd.read_csv("../../data/companyData/stockWeather_annual.csv").\
    drop(columns = {'Unnamed: 0'})

annualWeather = annualWeather[~annualWeather.temp_annualLast5.isna()].reset_index(drop = True)

annualWeather['date'] = pd.to_datetime(annualWeather['date'],
                                   format = "%Y%m%d")

annualWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(annualWeather.dtypes)
annualWeather.head()

In [ ]:
allWeather = pd.read_csv("../../data/companyData/stockWeather_zipQuarterQuants.csv").\
    drop(columns = {'Unnamed: 0'})

allWeather = allWeather[~allWeather.temp_zipQuarterLast5.isna()].reset_index(drop = True)

allWeather['date'] = pd.to_datetime(allWeather['date'],
                                   format = "%Y-%m-%d")

allWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(allWeather.dtypes)
allWeather.head()

In [ ]:
stocksWithControlsWeather = stocksWithControls.merge(allWeather).merge(annualWeather)
print(stocksWithControlsWeather.shape,allWeather.shape)

stocksWithControlsWeather.head()

In [ ]:
stocksWithControlsWeather.to_csv("../../data/companyData/stocksWithControlsWeather.csv")

In [ ]:
sum(stocksWithControlsWeather.RET.isna())

In [ ]:
stocksWithControlsWeather = pd.read_csv("../../data/companyData/stocksWithControlsWeather.csv").drop(columns = {'Unnamed: 0'})
stocksWithControlsWeather.head()

In [ ]:
sum(stocksWithControlsWeather.gvkey.isna())

--------------------

# Weather Data
First do this on the HQ zipcodes.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags.csv")

Do this across all zips, for the establishment records. We'll put this into a different format right after, and then change the columns and whatnot.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_allZips.csv").\
    drop(columns = {"Unnamed: 0", 'Unnamed: 0.1'})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags_allZips.csv")

Add in the few new definitions we've started on here: bins by week/month/quarter, and days 90+.

In [5]:
allWeather = pd.read_csv("../../data/companyData/latestExtremes.csv").\
    drop(columns = {"Unnamed: 0"})
allWeather.rename(columns = {'quarter': 'qtr'}, inplace = True)

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags_new.csv")

(1436864, 30)
2010    130624
2011    130624
2012    130624
2013    130624
2014    130624
2015    130624
2016    130624
2017    130624
2018    130624
2019    130624
Name: year, dtype: int64


Now do the same for the industry-specific weather.

In [ ]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\
allWeather_byInd = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_byInd.csv").\
    drop(columns = {"Unnamed: 0"})
'''[['famafrench','zipcode','yearQuarter', 
                                    'temp_ffquant_0.95','temp_indQuarterquant_0.95',
                                   'temp5Days_ffquant_0.95', 'temp5Days_indQuarterquant_0.95',
                                   'precip_ffquant_0.95', 'precip_indQuarterquant_0.95',
                                   'precip5Days_ffquant_0.95', 'precip5Days_indQuarterquant_0.95']]
'''
allWeather_byInd['year'] = allWeather_byInd.yearQuarter.str.slice(0,4).astype('int64')
allWeather_byInd['qtr']  = allWeather_byInd.yearQuarter.str.slice(5,6).astype('int64')
allWeather_byInd['yearQtr'] = allWeather_byInd.year + (allWeather_byInd.qtr - 1)/4

allWeather_byInd = allWeather_byInd.astype({'year':       'category',
                         'qtr':        'category',
                         'zipcode':    'category',
                         'famafrench': 'category'})

changes['zipcode'] = changes['zipcode'].astype({'zipcode': 'int64'})

changes = changes.astype({'year':       'category',
                          'qtr':        'category',
                          'zipcode':    'category',
                          'famafrench': 'category'})

col = allWeather_byInd.pop("year")
allWeather_byInd.insert(0, col.name, col)

col = allWeather_byInd.pop("qtr")
allWeather_byInd.insert(0, col.name, col)


col = allWeather_byInd.pop("yearQtr")
allWeather_byInd.insert(0, col.name, col)

allWeather_byInd.drop(columns = {'yearQuarter'}, inplace = True)

print(allWeather_byInd.head())

In [ ]:
lag1 = allWeather_byInd.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[5:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)
lag1 = lag1.astype({'yearQtr':       'category'})

    
lag2 = allWeather_byInd.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[5:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)
lag2 = lag2.astype({'yearQtr':       'category'})


lag3 = allWeather_byInd.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[5:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)
lag3 = lag3.astype({'yearQtr':       'category'})


lag4 = allWeather_byInd.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[5:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)
lag4 = lag4.astype({'yearQtr':       'category'})


allWeather_byInd = allWeather_byInd.astype({'yearQtr':       'category'})


print(allWeather_byInd.shape)


allWeather_byInd.head()


'''allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

allWeather_byInd_withLags.year.value_counts()
'''

In [ ]:
allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

In [ ]:
allWeather_byInd_withLags.shape

In [ ]:
allWeather_byInd_withLags.to_csv("../../data/companyData/allWeather_byInd_withLags.csv")

In [ ]:
del allWeather_byInd
del lag1
del lag2
del lag3
del lag4
gc.collect()

In [ ]:
allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv")
allWeather_byInd_withLags.head()

Now get the streak data. Make sure it's 0/1 for whether there was a streak.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/stockWeather_zipQuarterQuants.csv").\
    drop(columns = {'Unnamed: 0'})

allWeather = allWeather[~allWeather.temp_zipQuarterLast5.isna()].reset_index(drop = True)

allWeather['date'] = pd.to_datetime(allWeather['date'],
                                   format = "%Y-%m-%d")

allWeather['hotStreak']   = (allWeather['temp_zipQuarterLast5'] == 5)*1
allWeather['wetStreak'] = (allWeather['precip_zipQuarterLast5'] == 5)*1

allWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(allWeather.dtypes)

allWeather['year'] = allWeather.date.dt.year
allWeather['quarter'] = allWeather.date.dt.quarter


allWeather.drop(columns = {'date','temp_zipQuarterLast5','precip_zipQuarterLast5'}, inplace = True)

streaks = allWeather.groupby(['zipcode','year','quarter']).sum().reset_index()

streaks['hotStreak'] = (streaks['hotStreak'] > 0)*1
streaks['wetStreak'] = (streaks['wetStreak'] > 0)*1

streaks.rename(columns = {'quarter': 'qtr'}, inplace = True)
streaks.head()


In [ ]:
streaks['yearQtr'] = streaks.year + (streaks.qtr - 1)/4

col = streaks.pop("yearQtr")
streaks.insert(0, col.name, col)

lag1 = streaks.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = streaks.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = streaks.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = streaks.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(streaks.shape)

streaks_withLags = streaks.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(streaks_withLags.year.value_counts())

streaks_withLags.to_csv("../../data/companyData/streaks_withLags.csv")

In [ ]:
streaks_withLags.head()

# Locations
Create a separate definition of weather based not on HQ but on employee-weighted establishment footprint.

In [3]:
fractions = pd.read_csv('../../data/companyData/fractionEmployees_byEstablishment.csv').\
    drop(columns = {"Unnamed: 0", 'latitude','longitude'}).rename(columns = {'archive_version_year': 'year',
                                                    'parent_number': 'abi'})

fractions['year']    = fractions.year.astype('int64')
fractions['zipcode'] = fractions.zipcode.astype('int64')
fractions.head()

gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

print(gvKey_abiLinkingTable.abi)

gvKey_abiLinkingTable.head()

fractions = fractions[['year','abi','zipcode','locationFracOfEmployees']].merge(gvKey_abiLinkingTable[['abi','gvkey']])

fractions = fractions.astype({'year':       'category',
                           'zipcode':    'category'})

fractions.head()

0       402180222
1       397759739
2       400700704
3       218548014
4       441435880
          ...    
8394    432764101
8395    416920671
8396    739118540
8397    227688843
8398    488766353
Name: abi, Length: 8399, dtype: int64


,year,abi,zipcode,locationFracOfEmployees,gvkey
0,2003,100537,24541,0.047704,3937
1,2003,100537,24540,0.572451,3937
2,2003,100537,27828,0.357782,3937
3,2003,100537,27804,0.017889,3937
4,2003,100537,27834,0.001789,3937


In [6]:
fractions.year

0           2003
1           2003
2           2003
3           2003
4           2003
            ... 
14770031    2021
14770032    2021
14770033    2021
14770034    2021
14770035    2021
Name: year, Length: 14770036, dtype: category
Categories (19, int64): [2003, 2004, 2005, 2006, ..., 2018, 2019, 2020, 2021]

In [29]:
fractions_byZip = fractions[['gvkey','year','zipcode','locationFracOfEmployees']]
fractions_byZip['year'] =  fractions_byZip['year'].astype('int64')

fractions_byZip = fractions_byZip[(fractions_byZip.year > 2008) & (fractions_byZip.year < 2020)]

print(fractions_byZip.shape)

fractions_byZip.to_csv("../../data/companyData/fractions_byZip.csv")

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(9065033, 4)


In [ ]:
fractionsWithWeather = fractions.merge(allWeather_withLags_allZips) 
fractionsWithWeather.drop(columns = {'abi','zipcode'}, inplace = True)

print(fractionsWithWeather.shape)
fractionsWithWeather.head()

In [ ]:
fractionsWithWeather[fractionsWithWeather.gvkey == 1004]

In [ ]:
del allWeather_withLags
del fractions
del gvKey_abiLinkingTable
gc.collect()

In [ ]:
for col in fractionsWithWeather.columns[4:]:
    fractionsWithWeather[col] = fractionsWithWeather[col] * fractionsWithWeather.locationFracOfEmployees

In [ ]:
g = fractionsWithWeather.groupby(['gvkey','year','qtr']).sum().reset_index()
g.drop(columns = {'locationFracOfEmployees'}, inplace = True)

for colname in g.columns[3:]:
    g.rename(columns = {colname: 'empWt_' + colname}, inplace = True)

g.head()

In [ ]:
g.to_csv("../../data/companyData/weatherByEstablishment.csv")

In [ ]:
establishmentZips = fractions.zipcode.unique()
len(establishmentZips)

## create the original weather with lags dataset

In [40]:
streaks_withLags = pd.read_csv("../../data/companyData/streaks_withLags.csv").\
    drop(columns = {'Unnamed: 0', 'yearQtr'})

g = pd.read_csv("../../data/companyData/weatherByEstablishment.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})

averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})

allWeather_withLags2 = pd.read_csv("../../data/companyData/allWeather_withLags_new.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})


averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 



streaks_withLags          = streaks_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

allWeather_withLags       = allWeather_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

allWeather_withLags2      = allWeather_withLags2.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

averages                  = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})

allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})

allWeather_byInd_withLags = allWeather_byInd_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

Create direct effects database. Merge weather to full cstat database.

In [41]:
allWeather_withLags.zipcode

0          1001
1          1001
2          1001
3          1001
4          1001
          ...  
167715    99403
167716    99403
167717    99403
167718    99403
167719    99403
Name: zipcode, Length: 167720, dtype: category
Categories (4193, int64): [1001, 1013, 1085, 1089, ..., 99352, 99354, 99362, 99403]

In [39]:
'''changes.zipcode = changes.zipcode.astype('int64')

changes.columns

# changes['zipcode']  = changes['zipcode'].astype('int64')
changesWithWeather = changes.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages).merge(g)
print(changes.shape,changesWithWeather.shape)

changesWithWeather.indGroup.value_counts()


print(changesWithWeather.sic2.value_counts()[50:85])

changesWithWeather.to_csv("../../data/companyData/cstatWithWeather.csv")
'''

NameError: name 'changes' is not defined

Merge weather to the ig-cstat database.

In [42]:
igChanges = pd.read_csv("../../data/companyData/igData.csv").drop(columns = {'Unnamed: 0', 
        'parent_employee_size_code', 'location_employee_size_code', 'employeesAtLocation'})

fractions_byZip = pd.read_csv("../../data/companyData/fractions_byZip.csv").drop(columns = {'Unnamed: 0'})
fractions_byZip = fractions_byZip[fractions_byZip.gvkey.isin(list(igChanges.gvkey.unique())) & \
                                  fractions_byZip.zipcode.isin(list(igChanges.zipcode.unique())) ]

fractions_byZip = fractions_byZip.groupby(['year','zipcode','gvkey']).sum().reset_index()

print(igChanges.shape)

igChanges = igChanges.merge(fractions_byZip)

print(igChanges.shape)

(241355, 61)
(134423, 62)


In [43]:
fractions_byZip.shape


(1762738, 4)

In [44]:
igChangesWithWeather = igChanges.merge(allWeather_withLags).merge(allWeather_withLags2).merge(allWeather_byInd_withLags).merge(averages).merge(g).merge(streaks_withLags)
igChangesWithWeather.shape

(116342, 858)

In [45]:
igChangesWithWeather.to_csv("../../data/companyData/igWithWeather.csv")

In [33]:
igChangesWithWeather = pd.read_csv("../../data/companyData/igWithWeather.csv")

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
for col in igChangesWithWeather.columns:
    print(col)

In [34]:
igChangesWithWeather.shape

(125500, 723)

# Indirect
Introduce the SC Data.

In [ ]:
# this does a little bit of a test on the reporting requirements. 
# number 

'''c_linksTest = pd.read_csv("../../data/companyData/compustatSCLinked.csv")[['srcdate','gvkey','cgvkey']]
c_linksTest['year'] = c_linksTest.srcdate.astype('str').str.slice(0,4).astype('int64')

bs = c_linksTest[c_linksTest.year < 2014]
print("Customers per supplier, 1978-2013 Pd: ", len(bs.cgvkey.unique())/len(bs.gvkey.unique()))

bs2 = c_linksTest[c_linksTest.year > 2010]
print("Customers per supplier, Recent Pd: ", len(bs2.cgvkey.unique())/len(bs2.gvkey.unique()))'''


In [39]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv") # pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 1999][['year','gvkey','cgvkey','salecs']].\
    rename(columns = {'cgvkey': 'customer_gvkey','gvkey': 'supplier_gvkey'})

c_links['year'] = pd.to_datetime(c_links.year, format = '%Y')


c_links.head()

,year,supplier_gvkey,customer_gvkey,salecs
70,2002-01-01,1013,2136,111.056
71,2004-01-01,1013,2136,104.312
72,2005-01-01,1013,2136,146.000
73,2006-01-01,1013,2136,205.000
74,2007-01-01,1013,2136,236.000


In [40]:
supplierCombos = c_links[['supplier_gvkey', 'customer_gvkey']].drop_duplicates().reset_index(drop = True)

print(supplierCombos.shape)

supplierCombos.head()

(16812, 2)


,supplier_gvkey,customer_gvkey
0,1013,2136
1,1013,9899
2,1021,61494
3,1021,25880
4,1048,11552


We'll follow Barrot Sauvagnat in assuming that a supplier relationship holds for every year between the first and last year in which a customer is reported. This is going to take a little bit of work. We'll try it like this: 
- subset dataframe to a specific supplier-customer pair
- fill in data for every year that's missing

Then, apply this row-wise to all rows of the unique supplierCombos df above using: https://stackoverflow.com/questions/61942138/apply-function-row-wise-to-pandas-dataframe

In [41]:
def fillYear(supplier, customer, scData = c_links):
    c_linksTemp = scData[(scData.supplier_gvkey == supplier) & \
                      (scData.customer_gvkey == customer)].reset_index(drop = True)
    
    # if there are na values and non-na values for the same supplier-cust combination, then 
    # select for only the non-na values, by (1) replacing na with negative, (2) 1
    c_linksTemp['salecs'] = c_linksTemp['salecs'].fillna(-5)
    c_linksTemp = c_linksTemp.loc[c_linksTemp.reset_index().groupby(['year','supplier_gvkey', 'customer_gvkey'])['salecs'].idxmax()]

    
    # now: find the start and end of the data series
    first = c_linksTemp.year.min()
    last  = c_linksTemp.year.max()

    c_linksTemp = c_linksTemp.set_index('year') 

    c_linksTemp = c_linksTemp.reindex(pd.date_range(first, last, freq = 'YS')).\
        reset_index().rename(columns = {'index': 'year'})

    # and impute all values within the series
    c_linksTemp = c_linksTemp.groupby(c_linksTemp.year.dt.time).ffill()
    
    return(c_linksTemp)

Show that this works for one of the supplier rows.

In [42]:
fillYear(supplierCombos.supplier_gvkey[0], supplierCombos.customer_gvkey[0])

,year,supplier_gvkey,customer_gvkey,salecs
0,2002-01-01,1013.0,2136.0,111.056
1,2003-01-01,1013.0,2136.0,111.056
2,2004-01-01,1013.0,2136.0,104.312
3,2005-01-01,1013.0,2136.0,146.000
4,2006-01-01,1013.0,2136.0,205.000
5,2007-01-01,1013.0,2136.0,236.000
6,2008-01-01,1013.0,2136.0,240.000
7,2009-01-01,1013.0,2136.0,176.000
8,2010-01-01,1013.0,2136.0,146.000


Now do it for all rows.

In [43]:
start = time.time()
print(c_links.shape)
c_linksImpd_list = supplierCombos.apply(lambda row: fillYear(row['supplier_gvkey'], row['customer_gvkey']), axis = 1)
c_linksImpd_df   = pd.concat(list(c_linksImpd_list))
print(c_linksImpd_df.shape)
print(time.time() - start)

c_linksImpd_df['year'] = c_linksImpd_df.year.dt.year

(65270, 4)
(68771, 4)
69.66637182235718


We had converted some of the na sales values to -5 so that we could deal with duplicated values, by choosing the larger of said values. Switch back to nan so that we are not thrown off when we look for biggest supplier.

In [44]:
c_linksImpd_df.loc[c_linksImpd_df.salecs == -5, 'salecs'] = float('nan')

In [45]:
c_linksImpd_df.salecs

0    111.056
1    111.056
2    104.312
3    146.000
4    205.000
      ...   
1    179.284
0     34.418
0     25.334
0    283.318
1    316.116
Name: salecs, Length: 68771, dtype: float64

In [46]:
industries = pd.read_csv("../../data/companyData/gvkeyIndustries.csv").drop(columns = {'Unnamed: 0'})
print(industries)

        gvkey   indGroup
0        1004  wholesale
1        1010       manu
2        1013       manu
3        1019   services
4        1021       manu
...       ...        ...
25236  345920       manu
25237  345980     retail
25238  347085   services
25239  351491       manu
25240  351590       manu

[25241 rows x 2 columns]


In [47]:
c_links = c_linksImpd_df.copy()

print(c_links.shape)

print(c_links.head())

industries.columns = ['customer_gvkey','customer_ind']

c_links = c_links.merge(industries)

industries.columns = ['supplier_gvkey','supplier_ind']

c_links = c_links.merge(industries)
print(c_links.head(), c_links.shape)


c_links.to_csv("../../data/companyData/c_links.csv")


(68771, 4)
   year  supplier_gvkey  customer_gvkey   salecs
0  2002          1013.0          2136.0  111.056
1  2003          1013.0          2136.0  111.056
2  2004          1013.0          2136.0  104.312
3  2005          1013.0          2136.0  146.000
4  2006          1013.0          2136.0  205.000
   year  supplier_gvkey  customer_gvkey   salecs        customer_ind  \
0  2002          1013.0          2136.0  111.056  transportUtilities   
1  2003          1013.0          2136.0  111.056  transportUtilities   
2  2004          1013.0          2136.0  104.312  transportUtilities   
3  2005          1013.0          2136.0  146.000  transportUtilities   
4  2006          1013.0          2136.0  205.000  transportUtilities   

  supplier_ind  
0         manu  
1         manu  
2         manu  
3         manu  
4         manu   (66561, 6)


Let's see how all this translates into different industries. Check how many times different industries show up.

The full count will be roughly 4x whatever is below, assuming we can get a match for roughly all of them (which we should be able to).

In [49]:
c_links[c_links.year > 2009].supplier_ind.value_counts()

manu                  15682
finance                8270
services               4294
mining                 2864
transportUtilities     1950
wholesale               802
construction            265
retail                  172
agForFish                62
Name: supplier_ind, dtype: int64

Now see if it's common to have one in and one out of the industries of interest. 

For now, let's keep all the different industry types.

We can always filter later if we need to.

In [50]:
#########################
# get data and reset columns 
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns



'''
# merge in customer information
gvKey_abiLinkingTable.columns = customer_columns

print(c_links.shape)
c_linksMerge1 = c_links.merge(gvKey_abiLinkingTable, on ='customer_gvkey')
print(c_links.shape,c_linksMerge1.shape)
'''



#########################
# and merge in supplier 
gvKey_abiLinkingTable.columns = supplier_columns

print(c_links.shape)
c_linksMerge2 = c_links.merge(gvKey_abiLinkingTable, on ='supplier_gvkey')
print(c_linksMerge2.shape)

c_linksMerge2.to_csv("../../data/companyData/clinks_IG_selected.csv")

(66561, 6)
(46937, 10)


In [51]:
sum(c_linksMerge2.year > 2009)

27172

In [52]:
c_linksMerge2.head()

,year,supplier_gvkey,customer_gvkey,salecs,customer_ind,supplier_ind,supplier_cstatCompanies,supplier_igCompanies,supplier_delete,supplier_abi
0,2002,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
1,2003,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
2,2004,1013.0,2136.0,104.312,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
3,2005,1013.0,2136.0,146.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
4,2006,1013.0,2136.0,205.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129


This is probably because: (1) companies are not in North America, or (2) companies are not in the physical goods industries we're interested in. We can verify this though: look at c_links where both the customer and supplier are in the dataset of interest.

In [53]:
chq     = pd.read_csv("../../data/chq.csv",dtype={'cstatZipcode': 'object'}).drop(columns = {'Unnamed: 0'})

c_linkTest = c_links[c_links.customer_gvkey.isin(chq.gvkey.unique()) & \
                     c_links.supplier_gvkey.isin(chq.gvkey.unique())]

print("Percent of firms with a match: ", c_linksMerge2.shape[0]/c_linkTest.shape[0])

Percent of firms with a match:  0.9528614060374754


In [54]:
c_linksMerge2.head()

,year,supplier_gvkey,customer_gvkey,salecs,customer_ind,supplier_ind,supplier_cstatCompanies,supplier_igCompanies,supplier_delete,supplier_abi
0,2002,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
1,2003,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
2,2004,1013.0,2136.0,104.312,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
3,2005,1013.0,2136.0,146.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
4,2006,1013.0,2136.0,205.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129


It's entirely possible that we have too small of a sample from the 2010s alone. Let's just try it though and see how it goes.

First, make a sample with the companies on one year of either side of when it reports another customer.

In [55]:
def makeOneEitherSide(df): 
    yrPlus1 = df.copy(); yrPlus1['year'] += 1
    # yrPlus2 = df.copy(); yrPlus2['year'] += 1
    # yrPlus3 = df.copy(); yrPlus3['year'] += 1
    
    yrMinus1 = df.copy(); yrMinus1['year'] -= 1
    # yrMinus2 = df.copy(); yrMinus2['year'] -= 1
    # yrMinus3 = df.copy(); yrMinus3['year'] -= 1
    
    all = pd.concat([yrPlus1,yrMinus1]) # pd.concat([yrPlus1,yrPlus2,yrPlus3,yrMinus1,yrMinus2,yrMinus3])
    
    return(all)

In [56]:
c_linksMerge2.columns

Index(['year', 'supplier_gvkey', 'customer_gvkey', 'salecs', 'customer_ind',
       'supplier_ind', 'supplier_cstatCompanies', 'supplier_igCompanies',
       'supplier_delete', 'supplier_abi'],
      dtype='object')

In [57]:
scTableSuppliers = c_linksMerge2.copy()[['year','supplier_gvkey','supplier_abi','supplier_ind']].drop_duplicates()

print(scTableSuppliers.shape)
# allSupplierData = makeOneEitherSide(scTableSuppliers)
allSupplierData         = scTableSuppliers.copy()
allSupplierData.columns = ['year','gvkey','abi','ind']


allAbi = allSupplierData.abi.drop_duplicates() # allCustomerData.abi.append(

########
hqsOnly = pd.read_csv("../../data/ig_uniqueHQs.csv").drop(columns = {'Unnamed: 0'})

hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

hqRelevant = hq[hq.abi.isin(allAbi)]


allSupplierData = allSupplierData.merge(hqRelevant).drop_duplicates()

print(allSupplierData.head(), allSupplierData.shape)

allSupplierData.to_csv("../../data/companyData/allSupplierData.csv")

(19295, 4)
   year   gvkey      abi   ind ticker                     company state  \
0  2003  1013.0  7523129  manu    NaN  ADC TELECOMMUNICATIONS INC    MN   
1  2004  1013.0  7523129  manu    NaN  ADC TELECOMMUNICATIONS INC    MN   
2  2005  1013.0  7523129  manu    NaN  ADC TELECOMMUNICATIONS INC    MN   
3  2006  1013.0  7523129  manu    NaN  ADC TELECOMMUNICATIONS INC    MN   
4  2007  1013.0  7523129  manu    NaN  ADC TELECOMMUNICATIONS INC    MN   

           city       address_line_1  zipcode  latitude  longitude  \
0  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
1  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
2  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
3  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
4  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   

   parent_employee_size_code  location_employee_size_code  employeesAtLocation  
0                      250.0        

Previously we had done this with the customers as well, but we lose some percentage of the observations if we again try to match on the IG data, so for now just focus on the suppliers.

## Find Customer and Supplier pairings and merge with change data
### Can pick up here

In [48]:
allSupplierData = pd.read_csv("../../data/companyData/allSupplierData.csv").\
    drop(columns = ['Unnamed: 0'])[['gvkey', 'ind', 'year','zipcode']]
print(allSupplierData.shape, allSupplierData.columns)

(15261, 4) Index(['gvkey', 'ind', 'year', 'zipcode'], dtype='object')


In [49]:
allSupplierData.head()

,gvkey,ind,year,zipcode
0,1013.0,manu,2003,55344
1,1013.0,manu,2004,55344
2,1013.0,manu,2005,55344
3,1013.0,manu,2006,55344
4,1013.0,manu,2007,55344


## Get first-hop SC data

In [42]:
c_links = pd.read_csv("../../data/companyData/clinks_IG_selected.csv").drop(columns = {'Unnamed: 0'})

print(c_links.shape)

c_links.head()

(46937, 10)


,year,supplier_gvkey,customer_gvkey,salecs,customer_ind,supplier_ind,supplier_cstatCompanies,supplier_igCompanies,supplier_delete,supplier_abi
0,2002,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
1,2003,1013.0,2136.0,111.056,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
2,2004,1013.0,2136.0,104.312,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
3,2005,1013.0,2136.0,146.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129
4,2006,1013.0,2136.0,205.000,transportUtilities,manu,adc telecommunications,adc telecommunications,NaN,7523129


In [43]:
c_links['suppliers'] = 1
custExp = c_links[['year', 'customer_gvkey', 'salecs','suppliers']].groupby(['year','customer_gvkey']).sum().\
    reset_index().rename(columns = {'salecs': 'totalExp'})

custExp.head()



,year,customer_gvkey,totalExp,suppliers
0,2000,1038.0,38.220,2
1,2000,1045.0,38.093,4
2,2000,1078.0,5.070,2
3,2000,1095.0,4.790,1
4,2000,1121.0,7.883,1


In [44]:
print("Number of firms with no exp information and multiple suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 1))
print("Number of firms with no exp information and >5 suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 5))


Number of firms with no exp information and multiple suppliers:  312
Number of firms with no exp information and >5 suppliers:  3


Most of these firms have expenditure information. We can look at:
    - Expenditure-weighted (just do equal shares if no exp information)
    - Largest supplier
    
    
Our focus is going to be on the economic data of the customers, so isolate for the customers here.

In [45]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()
print(customerDB.shape)

customerDB.head()

(46293, 6)


,year,gvkey,supplier_gvkey,salecs,totalExp,suppliers
0,2002,2136.0,1013.0,111.056,892.202,13
1,2002,2136.0,3275.0,8.398,892.202,13
2,2002,2136.0,10286.0,16.987,892.202,13
3,2002,2136.0,10420.0,229.158,892.202,13
4,2002,2136.0,14340.0,9.432,892.202,13


## Merge in supplier weather
Get the weather data.

In [46]:
streaks_withLags = pd.read_csv("../../data/companyData/streaks_withLags.csv").\
    drop(columns = {'Unnamed: 0', 'yearQtr'})

g = pd.read_csv("../../data/companyData/weatherByEstablishment.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})

averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})


averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 



streaks_withLags          = streaks_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

allWeather_withLags       = allWeather_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

averages                  = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})

allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})
allWeather_byInd_withLags = allWeather_byInd_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

In [47]:
streaks_withLags.head()

,zipcode,year,qtr,hotStreak,wetStreak,lag1_hotStreak,lag1_wetStreak,lag2_hotStreak,lag2_wetStreak,lag3_hotStreak,lag3_wetStreak,lag4_hotStreak,lag4_wetStreak
0,1001,2010,1,1,0,0,0,0,0,0,0,0,0
1,1001,2010,2,0,0,1,0,0,0,0,0,0,0
2,1001,2010,3,1,0,0,0,1,0,0,0,0,0
3,1001,2010,4,0,0,1,0,0,0,1,0,0,0
4,1001,2011,1,0,0,0,0,1,0,0,0,1,0


The weather data, for now at least, is only back until 2009. We can potentially change this but for now it fits the ``last ten years" theme.

In [50]:
suppliersWithWeather = allSupplierData[allSupplierData.year > 2009].merge(streaks_withLags).merge(allWeather_withLags).merge(averages).merge(allWeather_byInd_withLags).merge(g)
print("supplier: ", suppliersWithWeather.shape)

suppliersWithWeather.to_csv("../../data/companyData/suppliersWithWeather.csv")

supplier:  (1425020, 672)


In [51]:
suppliersWithWeather.head()

,gvkey,ind,year,zipcode,qtr,hotStreak,wetStreak,lag1_hotStreak,lag1_wetStreak,lag2_hotStreak,...,empWt_lag4_precip_zipQuarterquant_1x5Yrs,empWt_lag4_temp_annualquant_0.95,empWt_lag4_temp_annualquant_1x5Qtrs,empWt_lag4_temp_annualquant_1x5Yrs,empWt_lag4_temp_zipquant_0.95,empWt_lag4_temp_zipquant_1x5Qtrs,empWt_lag4_temp_zipquant_1x5Yrs,empWt_lag4_temp_zipQuarterquant_0.95,empWt_lag4_temp_zipQuarterquant_1x5Qtrs,empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
1,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
2,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
3,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
4,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0


## Any Supplier
Focus on weather of all suppliers. First rename the suppliers data.

In [52]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x)        |
                                                 ('gvkey' in x)      |
                                                 ('famafrench' in x) |
                                                 ('Streak' in x)     | 
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[4:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey',
                                   'famafrench': 'supplier_famafrench'},inplace = True)    

print(suppliers_toMerge.columns)


Index(['supplier_gvkey', 'year', 'qtr', 'hotStreak', 'supplier_wetStreak',
       'supplier_lag1_hotStreak', 'supplier_lag1_wetStreak',
       'supplier_lag2_hotStreak', 'supplier_lag2_wetStreak',
       'supplier_lag3_hotStreak',
       ...
       'supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs',
       'supplier_empWt_lag4_temp_annualquant_0.95',
       'supplier_empWt_lag4_temp_annualquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_annualquant_1x5Yrs',
       'supplier_empWt_lag4_temp_zipquant_0.95',
       'supplier_empWt_lag4_temp_zipquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_zipquant_1x5Yrs',
       'supplier_empWt_lag4_temp_zipQuarterquant_0.95',
       'supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs'],
      dtype='object', length=670)


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Get the change data and merge with the customers in the customer-supplier dataset.

In [53]:
changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.columns)
customers = changes.merge(customerDB)
print(customers.shape)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue', 'cik',
       'costat', 'add1', 'add2', 'addzip', 'city', 'sic', 'state',
       'assetsLast', 'netIncomeLast', 'totalRevenueLast', 'costGoodsSoldLast',
       'totalInvLast', 'incomeChange', 'revenueChange', 'costChange',
       'inventoryChange', 'assetsPrev', 'assetsLagged', 'netIncomeLagged',
       'roa_lagged', 'famafrench', 'sic2', 'indGroup', 'earliestYear',
       'ageTercile', 'sizeTercile', 'profitTercile', 'datacqtr', 'datafqtr',
       'fyr', 'DATE'],
      dtype='object')
(124640, 47)


In [54]:
suppliersWithWeather.head()

,gvkey,ind,year,zipcode,qtr,hotStreak,wetStreak,lag1_hotStreak,lag1_wetStreak,lag2_hotStreak,...,empWt_lag4_precip_zipQuarterquant_1x5Yrs,empWt_lag4_temp_annualquant_0.95,empWt_lag4_temp_annualquant_1x5Qtrs,empWt_lag4_temp_annualquant_1x5Yrs,empWt_lag4_temp_zipquant_0.95,empWt_lag4_temp_zipquant_1x5Qtrs,empWt_lag4_temp_zipquant_1x5Yrs,empWt_lag4_temp_zipQuarterquant_0.95,empWt_lag4_temp_zipQuarterquant_1x5Qtrs,empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
1,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
2,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
3,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0
4,1013.0,manu,2010,55344,1,1,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.0


In [55]:
allSuppliersWithWeather = customers.merge(suppliers_toMerge)

In [56]:
allSuppliersWithWeather.to_csv("../../data/companyData/allSuppliersWithWeather.csv")

### stock data
merge in weather at the location of any supplier. First, get the supplier names only.

In [ ]:
allSupplierData = pd.read_csv("../../data/companyData/allSupplierData.csv").\
    drop(columns = ['Unnamed: 0'])[['gvkey', 'famafrench', 'year','zipcode']]
print(allSupplierData.shape, allSupplierData.columns)

allSupplierData.head()

And merge with the daily weather data.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/stockWeather_zipQuarterQuants.csv").\
    drop(columns = {'Unnamed: 0'})

allWeather = allWeather[~allWeather.temp_zipQuarterLast5.isna()].reset_index(drop = True)

allWeather['date'] = pd.to_datetime(allWeather['date'],
                                   format = "%Y-%m-%d")

allWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(allWeather.dtypes)
allWeather.head()


allWeather['year'] = allWeather.date.dt.year

supplierWeather = allSupplierData.merge(allWeather)

In [ ]:
supplierWeather.head()

In [ ]:
supplierWeather.drop(columns = {'zipcode'},inplace = True)
supplierWeather.rename(columns = {'gvkey': 'supplier_gvkey','famafrench': 'supplier_famafrench'},inplace = True)

Now get the actual stock data.

In [ ]:
with open('../../data/stockReturns.pkl', 'rb') as f:
    stocks = pkl.load(f)[['date','gvkey','RET']]

stocks = stocks[stocks.date.dt.year > 2008]

stocks['qtr']  = stocks.date.dt.quarter
stocks['year'] = stocks.date.dt.year

stocks = stocks[~stocks.gvkey.isna()]
stocks['gvkey'] = stocks['gvkey'].astype(int)
print(stocks.shape)

stocks.head()

The controls data.

In [ ]:
igChanges = pd.read_csv("../../data/companyData/igData.csv").\
    drop(columns = {'Unnamed: 0'})
igChanges.head()

companyControls = igChanges[['gvkey','year','qtr','famafrench','ageTercile','sizeTercile','profitTercile','zipcode']]
companyControls.head()

And merge it all together.

In [ ]:
customerDB['gvkey'] = customerDB.gvkey.astype('int64')

In [ ]:
supplierStocks = customerDB.merge(companyControls).merge(stocks).merge(supplierWeather)
print(supplierStocks.shape)

supplierStocks.head()

In [ ]:
supplierStocks.to_csv("../../data/companyData/supplierStocks.csv")

## Biggest Supplier
Focus on weather of biggest supplier.

First find the max by supplier. Add back in any rows with only 1 supplier.

In [ ]:
customerDB.shape

In [ ]:
# https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
idx = customerDB.groupby(['year','gvkey']).salecs.\
    transform(max) == customerDB.salecs
largestSuppliers = customerDB[idx].reset_index(drop = True)
print(largestSuppliers.shape)

# find companies who only have one other supplier
singleSuppliers = customerDB[customerDB.suppliers == 1].reset_index(drop = True)
print(singleSuppliers.shape)

# find largest suppliers of different companies
largestSuppliers = largestSuppliers.append(singleSuppliers).drop_duplicates()
print(largestSuppliers.shape)



In [ ]:
largestSuppliers.head()

Merge in the change data for that gvkey.

In [ ]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('famafrench' in x) |
                                                 ('Streak' in x)     |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[4:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey',
                                   'famafrench': 'supplier_famafrench'},inplace = True)    

print(suppliers_toMerge.columns)


'''suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].\
    rename(columns = {'gvkey': 'supplier_gvkey',
                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',
                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})'''

In [ ]:
changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.columns)
customers = changes.merge(largestSuppliers) # gvkey is on the customer here
print(customers.shape)

In [ ]:
largestSuppliersWithWeather = customers.merge(largestSuppliers[['year', 'gvkey', 'supplier_gvkey']]).merge(suppliers_toMerge)
largestSuppliersWithWeather.shape

In [ ]:
for col in largestSuppliersWithWeather.columns:
    print(col)

In [ ]:
largestSuppliersWithWeather.to_csv("../../data/companyData/largestSuppliersWithWeather.csv")

## Sales-Weighted Average
If a company doesn't have sales-specific information, then assume equal shares. This doesn't happen for too many of the companies, thankfully.

In [ ]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()

customerDB['salesWeight'] = customerDB.salecs/customerDB.totalExp

customerDB.fillna(1, inplace = True)

Now merge this with the supplier weather data, and use the sales weights to find a sales-weighted average of the weather conditions for the suppliers.

In [ ]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[3:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey'},inplace = True)    


In [ ]:
suppliers_toMerge.head()

For each of the supplier weather columns, multiply the variable by the fraction of sales attributable to that relationship.

In [ ]:
supplierWeather = customerDB[['year','gvkey','supplier_gvkey','salesWeight']].merge(suppliers_toMerge)

for col in supplierWeather.columns[7:]:
        supplierWeather[col]   = supplierWeather.salesWeight*supplierWeather[col]
        
        

supplierWeather.drop(columns = {'supplier_gvkey','salesWeight'}, inplace = True)


print(supplierWeather.head())



# [['year','qtr','gvkey','supplier_tmax_quant_1.0','supplier_precip_quant_1.0']]

In [ ]:
supplierWtdAvgWeather = supplierWeather.groupby(['year','qtr','gvkey']).sum().reset_index().drop_duplicates()

In [ ]:
supplierWtdAvgWeather.gvkey.unique()

Merge the supplier weighted average weather data with the customer data that has weather as well.

In [ ]:
customersWithWeather.head()

In [ ]:
wtdAvgSuppliers = customersWithWeather.merge(supplierWtdAvgWeather)

wtdAvgSuppliers.head()

In [ ]:
wtdAvgSuppliers.shape

In [ ]:
wtdAvgSuppliers.to_csv("../../data/companyData/wtdAvgSuppliers.csv")

In [ ]:
wtdAvgSuppliers.head()

In [ ]:
wtdAvgSuppliers.columns[wtdAvgSuppliers.columns.str.contains('Tercile')]